# Running Training and Prediction Pipeline
---
This notebook provides all the commands to reproduce the results of training the models, and prediction on the full corpus.

This process does not have to be done to update the inventory, but simply to reproduce the reported results, (this is the process used to produce them in the first place).

This pipeline has the following steps:

*   Split the manually curated datasets
*   Train all models on the classificaiton and NER tasks
*   Select the best model for each task
*   Evaluate all models for each task on their test sets
*   Perform classification of full corpus
*   Run NER model on predicted biodata resource papers
*   Extract URLs from predicted positives
*   Process the predicted names
*   Perform automated initial deduplication
*   Flag the inventory for selective manual review

### ***Warning***:

Running the full pipeline trains many models, and their "checkpoint" files are quite large (~0.5GB per model, ~15GB in total). Simply running prediction requires much less resources, including storage space.

### Other use-cases

If you want to compare a new model to the previously compared models, you can add another row to `config/models_info.tsv`. This pipeline will train this model and compare it to the others. If the other trained model checkpoint files are still present from a previous run, they will not be re-trained during the process.

# Setup
---
### Mount Drive

First, mount Google Drive to have access to files necessary for the run:


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/GitHub/inventory_2022/

Run the make target to install Python and R dependencies.

In [ ]:
! make setup

# Obtaining Fine-tuned models

All fine-tuned models have been archived. They can be optionally downloaded using the following cell. This cell also splits the training data first so that Snakemake will not automatically retrain the models (the training data is an input to the models, so if it is split after downloading, Snakemake will think the models are out of date).

In [ ]:
# Split the labeled data sets
! snakemake -s snakemake/train_predict.smk --configfile config/train_predict.yml -c 1 --until split_classif_data
! snakemake -s snakemake/train_predict.smk --configfile config/train_predict.yml -c 1 --until split_ner_data

# Create output directory
! mkdir -p out/

# Download models (may take several minutes)
! git lfs install
! git clone https://huggingface.co/globalbiodata/inventory_2022_all_models

# Move models to proper directory and delete unused files
! mv inventory_2022_all_models/classification_models/ out/classif_train_out
! mv inventory_2022_all_models/ner_models/ out/ner_train_out
! rm -rf inventory_2022_all_models
! rm -rf out/classif_train_out/best
! rm -rf out/ner_train_out/best

# Running the pipeline
---
Now, we are ready to run the pipeline

## Previewing what has to be done.

The following can be run to get a preview of what has to be done.

In [ ]:
! make dryrun_reproduction

## Run it

The following cell will run the entire pipeline described above. It takes a while, even with GPU acceleration. Without GPU it will take a very long time, if it is able to finish at all.

In [ ]:
! make train_and_predict

# Selective Manual Review

After running the initial pipeline, the inventory has been flagged for selective manual review.

The file to be reviewed is located at:

`out/original_query/for_manual_review/predictions.csv`

Review the flagged columns according to the instruction sheet ([doi: 10.5281/zenodo.7768363](https://doi.org/10.5281/zenodo.7768363)), then place the manually reviewed file in the following folder:

`out/original_query/manually_reviewed/`

The file must still be named `predictions.csv`


# Processing Manual Review

Next, further processing is performed on the manually reviewed inventory.

If you simply want to reproduce the original results (without manually reviewing the inventory or training models), you can copy the files that would have been generated to this point with the following commands. Otherwise, skip this code chunk.

In [ ]:
! mkdir -p out/original_query
! mkdir -p out/original_query/manually_reviewed/
! cp data/manually_reviewed_inventory.csv out/original_query/manually_reviewed/predictions.csv
! mkdir -p out/classif_train_out/combined_train_stats/
! cp data/classif_metrics/combined_train_stats.csv out/classif_train_out/combined_train_stats/combined_stats.csv
! mkdir -p out/classif_train_out/combined_test_stats/
! cp data/classif_metrics/combined_test_stats.csv out/classif_train_out/combined_test_stats/combined_stats.csv
! mkdir -p out/ner_train_out/combined_train_stats/
! cp data/ner_metrics/combined_train_stats.csv out/ner_train_out/combined_train_stats/combined_stats.csv
! mkdir -p out/ner_train_out/combined_test_stats/
! cp data/ner_metrics/combined_test_stats.csv out/ner_train_out/combined_test_stats/combined_stats.csv

To also skip processing of the manually reviewed file (such as getting metadata from EuropePMC), and just perform data analysis on the original output files, run the following code chunk:

In [ ]:
! mkdir -p out/original_query/processed_countries/
! cp data/final_inventory_2022.csv out/original_query/processed_countries/predictions.csv

Final analysis of the inventory compares the resources found to those in re3data and FAIRsharing. FAIRsharing requires login credentials to use their API. Before proceeding, please create an account at FAIRsharing, and enter your email address and password in the file `config/fairsharing_login.json`.

After manually reviewing the inventory or running the above code chunks to copy the previous files, final processing is performed with the below code chunk:

In [ ]:
! make process_manually_reviewed_original

# Results
---
Once the pipeline everythuing is complete, there are a few important output files


## Final inventory

The final inventory, including names, URLS, and metadata is found in the file:
*    `out/original_query/processed_countries/predictions.csv`

## Model training stats

The per-epoch training statistics for all models are in the files:

*    `out/classif_train_out/combined_train_stats/combined_stats.csv`
*    `out/ner_train_out/combined_train_stats/combined_stats.csv`

## Test set evaluation

Performance measures of the trained models on the test set are located in the files:

*    `out/classif_train_out/combined_test_stats/combined_stats.csv`
*    `out/ner_train_out/combined_test_stats/combined_stats.csv`

## Selected models
The name of the best models are in the files:

*    `out/classif_train_out/best/best_checkpt.txt`
*    `out/ner_train_out/best/best_checkpt.txt`

## Figures and analyses

Figures showing the model performances on the validation sets are present:

* `analysis/figures/class_val_set_performances.png`
* `analysis/figures/class_val_set_performances.svg`
* `analysis/figures/ner_val_set_performances.png`
* `analysis/figures/ner_val_set_performances.svg`

There are tables of all models' performance on the validation and test sets:

* `analysis/figures/combined_classification_table.docx`
* `analysis/figures/combined_ner_table.docx`

Figures of location data are also output:

* `analysis/figures/author_countries.png`
* `analysis/figures/ip_coordinates.png`
* `analysis/figures/ip_countries.png`

Figures/table on text mining potential:

* `analysis/figures/text_mining_potential_plot.png`
* `analysis/figures/text_mining_potential_plot.svg`
* `analysis/figures/text_mining_potential.csv`

Comparisons to re3data and FAIRsharing:

* `analysis/inventory_re3data_fairsharing_summary.csv`
* `analysis/venn_diagram_sets.csv`

Finally, some stats on the invenotry are saved:

* `analysis/analysed_metadata.txt`